In [9]:
import tensorflow as tf
import QGOpt as qgo
import matplotlib.pyplot as plt

In [10]:
dtype_real = tf.float64
dtype_complex = tf.complex128

def adj(A):
    
    return tf.linalg.matrix_transpose(tf.math.conj(A))

# random isometric matrix
q = tf.random.normal((2, 3, 2, 2), dtype=dtype_real)
q = qgo.manifolds.real_to_complex(q)
q, _ = tf.linalg.qr(q)

# random gradient
grad = tf.random.normal((2, 3, 2, 2), dtype=dtype_real)
grad = qgo.manifolds.real_to_complex(grad)

# random metric
L = tf.random.normal((2, 3, 2, 6, 2), dtype=dtype_real)
L = qgo.manifolds.real_to_complex(L)
metric = tf.einsum('qijn,qkln->qijkl', tf.math.conj(L), L)

# random tangent vector
v = tf.random.normal((2, 3, 2, 2), dtype=dtype_real)
v = qgo.manifolds.real_to_complex(v)

v = v - 0.5 * q @ (adj(q) @ v + adj(v) @ q)

v1 = tf.random.normal((2, 3, 2, 2), dtype=dtype_real)
v1 = qgo.manifolds.real_to_complex(v1)

v1 = 0.5 * q @ (adj(q) @ v1 + adj(v1) @ q)
print(v1 - 0.5 * q @ (adj(q) @ v1 + adj(v1) @ q))
print(tf.einsum('qij,qij->q', tf.math.conj(v), v1))
print(0.5 * q @ (adj(q) @ v + adj(v) @ q))

tf.Tensor(
[[[ 0.00000000e+00+2.77555756e-17j  2.77555756e-17+2.77555756e-17j]
  [ 0.00000000e+00+2.77555756e-17j -4.16333634e-17+0.00000000e+00j]
  [-6.93889390e-18-5.55111512e-17j  0.00000000e+00+0.00000000e+00j]]

 [[-2.77555756e-17+0.00000000e+00j  0.00000000e+00+2.77555756e-17j]
  [-2.77555756e-17-4.16333634e-17j  0.00000000e+00+5.55111512e-17j]
  [-5.55111512e-17+0.00000000e+00j -1.11022302e-16+2.77555756e-17j]]], shape=(2, 3, 2), dtype=complex128)
tf.Tensor([5.90792567e-18+0.74162223j 6.41542469e-16-3.95521883j], shape=(2,), dtype=complex128)
tf.Tensor(
[[[-1.04272406e-16-7.90748473e-17j -8.70562565e-17-1.69679714e-16j]
  [ 1.04195567e-16-1.26327159e-16j  3.01742294e-16+8.02293353e-17j]
  [ 8.54793772e-18-4.38103241e-17j -6.88627803e-17-1.98952399e-16j]]

 [[ 2.89500295e-18+1.01678282e-16j -3.31854466e-17+1.55372239e-17j]
  [ 1.15945617e-16-6.63180146e-17j  9.48557619e-18+1.16298052e-16j]
  [ 2.64351281e-16+1.50002351e-16j -7.38817158e-17+2.26025182e-16j]]], shape=(2, 3, 2), dty

### Manifold and Riemannian grad

In [11]:
manifold = qgo.any_metric_manifolds.StiefelManifold(retraction='cayley')
rgrad = manifold.egrad_to_rgrad(metric, q, grad)

### Projection on the nromal space

In [12]:
tf.linalg.norm(q @ (adj(q) @ rgrad + adj(rgrad) @ q))

<tf.Tensor: shape=(), dtype=complex128, numpy=(1.5582066677155852e-14+3.5471476669266133e-32j)>

### Test of the direction

In [13]:
vec = tf.einsum('qijkl,qkl->qij', metric, rgrad) - grad
vec1 = 0.5 * q @ (adj(q) @ vec + adj(vec) @ q)

print(0.5 * q @ (adj(q) @ vec - adj(vec) @ q) +\
      (tf.eye(3, dtype=dtype_complex) - q @ adj(q)) @ vec)

tf.reduce_sum(tf.math.conj(v) * vec, axis=(1, 2))

tf.Tensor(
[[[ 3.66433270e-14+1.47423316e-14j -7.87307912e-15-3.38249916e-16j]
  [ 1.41772382e-15-2.54320724e-14j  7.69079621e-15-3.00107965e-15j]
  [ 3.36043131e-15-1.80155522e-14j  4.54643441e-15+7.91787906e-15j]]

 [[-1.29771210e-14+9.51891370e-15j -1.86482398e-14-3.09303585e-14j]
  [-2.18111138e-14+5.26227487e-15j -2.11085109e-14+5.85933851e-14j]
  [-1.92511665e-15+1.09773867e-14j  4.70384944e-14+5.33510616e-15j]]], shape=(2, 3, 2), dtype=complex128)


<tf.Tensor: shape=(2,), dtype=complex128, numpy=array([1.06581410e-14+0.20645022j, 1.07469589e-13-0.33598092j])>

In [8]:
print(0.5 * q @ (adj(q) @ v - adj(v) @ q) +\
      (tf.eye(3, dtype=dtype_complex) - q @ adj(q)) @ v)
print(v)

tf.Tensor(
[[[-1.12474293-1.96475061j  0.41100072+0.28132027j]
  [ 2.33770681-1.40922325j  0.0385208 -0.57743326j]
  [-0.28712694-0.46864989j  1.0986696 +0.47547931j]]

 [[-0.02644441-0.73629405j -1.86688742-1.24069862j]
  [-0.64337275+0.24869449j -0.09699218+0.58586118j]
  [ 0.70149782-0.07009199j  0.65292322-0.80958282j]]], shape=(2, 3, 2), dtype=complex128)
tf.Tensor(
[[[-1.12474293-1.96475061j  0.41100072+0.28132027j]
  [ 2.33770681-1.40922325j  0.0385208 -0.57743326j]
  [-0.28712694-0.46864989j  1.0986696 +0.47547931j]]

 [[-0.02644441-0.73629405j -1.86688742-1.24069862j]
  [-0.64337275+0.24869449j -0.09699218+0.58586118j]
  [ 0.70149782-0.07009199j  0.65292322-0.80958282j]]], shape=(2, 3, 2), dtype=complex128)


In [8]:
vec

<tf.Tensor: shape=(2, 3, 2), dtype=complex128, numpy=
array([[[ 0.52688868+0.07712976j, -0.63055173+1.08787398j],
        [-1.32020732+0.15438808j,  0.38193662-2.62455906j],
        [ 0.2190537 +0.31066373j, -0.43265397+0.48473429j]],

       [[ 0.00780034+0.17175017j,  3.45913517+0.73193002j],
        [ 3.14132692-1.41358317j,  0.67045419-0.5225142j ],
        [ 1.58853854+0.5002948j ,  0.30822823-2.7322754j ]]])>

In [5]:
qgo.manifolds.stiefel_any_metric._to_complex_matrix(qgo.manifolds.stiefel_any_metric._to_real_matrix(q))

<tf.Tensor: shape=(2, 3, 2), dtype=complex128, numpy=
array([[[-0.04365868-0.6116606j , -0.37897955-0.59950323j],
        [ 0.60752004+0.14165557j, -0.29390455-0.1257153j ],
        [ 0.48254695+0.04434841j, -0.42942642+0.45867103j]],

       [[-0.56297839+0.77139332j,  0.10656735+0.07070004j],
        [-0.10613196+0.12112974j, -0.91556667+0.01866676j],
        [-0.00603807-0.24906787j,  0.06165343+0.37580977j]]])>

In [7]:
tf.linalg.eigvalsh(rgrad[0])

<tf.Tensor: shape=(24,), dtype=float64, numpy=
array([-3.63178195e-16, -1.77232651e-16, -1.03697802e-17,  4.16161000e-18,
        1.12646051e-16,  1.74623668e-16,  3.22575485e-16,  3.48443790e-16,
        5.07327600e-16,  6.59644558e-16,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00])>